In [18]:
class Dog:

    def __init__(self, name):
        self.name = name
        self.tricks = []    # creates a new empty list for each dog

    def add_trick(self, trick, *other_tricks):
        
        self.tricks.append(trick)
        
        for t in other_tricks:
            self.tricks.append(t)
        
        return self

In [16]:
d = Dog('Buddy').add_trick('shake', 'shake')

In [17]:
d.tricks

['shake', 'shake']